In [0]:
dbutils.widgets.text("p_data_date", "")
v_data_date = dbutils.widgets.get("p_data_date")

In [0]:
dbutils.secrets.listScopes()

In [0]:
dbutils.secrets.list(scope="Fomular1-scope")

In [0]:
fomular1_key= dbutils.secrets.get(scope="Fomular1-scope", key="formuladl1-key-account")

In [0]:
spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", fomular1_key)

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)])

drive_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                 StructField("driverRef", StringType(), True),
                                 StructField("number", IntegerType(), True),
                                 StructField("code", StringType(), True),
                                 StructField("name", name_schema),
                                 StructField("dob", DateType(), True),
                                 StructField("nationality", StringType(), True),
                                 StructField("url", StringType(), True)])


In [0]:
drive_df = spark.read.schema(drive_schema).json(f"abfss://raw@fomular1dl1.dfs.core.windows.net/{v_data_date}/drivers.json")
drive_df.show(
)

In [0]:
drive_df.schema

In [0]:
from pyspark.sql.functions import col, concat, lit, current_timestamp

final_df = drive_df.withColumnRenamed("driverId", "driver_id") \
                            .withColumnRenamed("driverRef", "driver_ref") \
                            .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                            .withColumn("ingestion_date", current_timestamp()) \
                            .drop(col("url"))
final_df.show()


In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")

In [0]:
%sql 

SELECT * from f1_processed.drivers